In [9]:
import pandas as pd
import numpy as np
import seaborn as sbn
import re
from matplotlib import pyplot


# Load the CSV and Name the Columns (I wasn't sure what a few of these were so I just dropped 'em')

In [14]:
# df = pd.read_csv('./assets/salaries.csv', names=['date','tmp0','title','tmp','tmp2','location','tmp23','tmp3','tmp34','tmp4','org_size','org_size_bucket','salary','tmp5','tmp6','experience', 'tmp7','tmp8','industry','tmp9', 'comments', 'tmp10'])
df = pd.read_csv('./assets/salaries.csv')

# df[['title','location','salary','experience']]

In [15]:
df

,Timestamp,Title,Location,Remote?,Org Size,Salary,Years of experience,Industry,Additional comments,Additional perks?
0,9/9/2020 22:17:07,Head of <redacted> technologies,"St. Paul, Minnesota",No,51 - 500,"US$165,000",NaN,NaN,NaN,NaN
1,9/20/2020 12:19:13,Senior Software Engineer,Porto Alegre/RS - Brazil,No,> 5000,US$ 18400 / year,17,Mediatech,NaN,NaN
2,9/19/2020 18:50:46,Senior Site Reliability Engineer,"Toronto, Ontario",No,501 - 1000,"CAD $174,000",21,Software/SaaS,NaN,"Options, hiring bonus, stock purchase plan, an..."
3,9/9/2020 20:25:55,Software Engineer,"Chantilly, VA",No,> 5000,98000,NaN,NaN,NaN,NaN
4,9/19/2020 21:01:23,Web Developer,"Canberra, ACT",Yes,11- 50,"$97,294",13,Marketing,The tech industry does exist outside of the US...,No
...,...,...,...,...,...,...,...,...,...,...
2046,2/23/2021 21:37:14,Software Engineer,Chicago Illinois,No,51 - 500,115000,10,"RegTech, FinTech",Black woman,5k
2047,2/23/2021 21:41:54,Senior Application Architect,Northern NJ,No,51 - 500,175000,24,Consulting,NaN,no
2048,2/23/2021 21:54:05,Senior Software Engineer,Atlanta,No,> 5000,130000,7,Automotive,Every salary bump has been diagonal move. I've...,13000
2049,2/23/2021 21:56:53,Design Lead,Remote,Yes,11- 50,65000,7,Software consultancy,NaN,Nope


In [17]:
# Turn 100k into 100,000 and remove commas
cleaned_salary = df['Salary'].str.replace('k','000').str.replace(',','')
# extract a number from the salaries if possible. Still won't work if people wrote like "one hundred thousand" 
# I guess that might be one change you could make to the survey is specify this field is in dollars and only allow a numeric input?
cleaned_salary = cleaned_salary.str.extract('(\d+)', expand=False)
cleaned_salary = cleaned_salary.astype(float)

# extract a numeric from the experience answer too if possible
cleaned_experience = df['Years of experience'].str.extract('(\d+)').astype(float)

df['salary'] = cleaned_salary
df['experience'] = cleaned_experience


# Take the first pre-comma chunk of text for location (so "Los Angeles, CA" and "Los Angeles" bot become "Los Angeles")
df['location'] = df['Location'].str.split(',').str[0]
# Remove non-alpha characters and lowercase to try and make these location names more standard
df['location'] = df['location'].fillna('').apply(lambda x: re.sub("[^a-zA-Z ]+", '', x, flags=re.UNICODE)).str.lower()
# Just fix DC manually
df.loc[df['location'] == 'washington', 'location'] = 'washington dc'

# Drop Some Bad Outliers

In [23]:
df = df[df['salary'] < 1000000]
df = df[df['experience'] < 100]

In [24]:
# this doesnt cover a lot of these titles, but you get the idea
def title_substrings(title, substrings):
    for substring in substrings:
        if substring in title:
            return True
    return False

def title_function(title):
    title = title.lower()
    if title_substrings(title, ['data', 'artificail', 'ml', 'scien']):
        return 'Data'
    elif title_substrings(title, ['front', 'javascript', 'react', 'ui']):
        return 'Front End'
    elif title_substrings(title, ['back']):
        return 'Back End'
    elif title_substrings(title, ['full','web', 'rails']):
        return 'Full Stack'
    elif title_substrings(title, ['mobile','ios', 'android']):
        return 'Mobile'
    elif title_substrings(title, ['ops','system', 'security', 'tool', 'network']):
        return 'Ops and Security'
    elif title_substrings(title, ['ux','user', 'design']):
        return 'UX and Design'
    elif title_substrings(title, ['manag','director', 'arch', 'president', 'cto', 'ceo', 'head', 'gm', 'coo', 'vp', 'chief']):
        return 'Management and Leadership'
    elif title_substrings(title, ['market', 'content', 'writ']):
        return 'Marketing and Content'
    elif title_substrings(title, ['test', 'quality', 'qa']):
        return 'QA and Testing'
    elif title_substrings(title, ['support']):
        return 'Support'
    elif title_substrings(title, ['product']):
        return 'Product'
    elif title_substrings(title, ['engineer', 'software', 'dev', 'program', 'tech']):
        return 'General Engineering'
    else:
        return 'Unknown'

df['title_categories'] = df['Title'].fillna('').apply(title_function)

# I don't know what a lot of these titles are, these are the titles I didn't categorize in any way...

In [25]:
df[df['title_categories'] == 'Unknown']['Title'].unique()

array(['SWE Intern', 'Community Advocate', 'Applications Analyst',
       'Solutions Analyst', 'Junior Salesforce Administrator ',
       'Lead Analyst', 'Lead SRE', 'Advanced Physicist',
       'Sr Business Analyis Consultant', 'Sr. SWE', 'Senior Consultant ',
       'Senior Business Analyst ', 'Red Team Lead', 'Associate ',
       'Senior SDM', 'Senior Sdet', 'SRE', 'Jr. Salesforce Administrator',
       'Senior FP&A Analyst', 'Staff 1', 'Senior Digital Strategist ',
       'ERP Consultant', 'Sales Strategy', 'Social Media Specialist',
       'Sr Analyst', 'Sr SDE', 'Team Lead', 'Senior Financial Analyst',
       'Lead Producer', 'z/os scheduller', 'Business Analyst ',
       'Senior SOC Analyst', 'CIO', 'Sr Sales Operations Analyst',
       'Salesforce Consultant', 'Senior SRE',
       'Senior Partnerships Associate', 'Server Storage Administrator',
       'Founder', 'Business consultant', 'Infrastructure Analyst',
       'Infosec.Consultant', 'Analytic Consultant', 'Consultant ',
 

In [26]:
df.to_csv('./assets/cleaned_data.csv')